Downloading: 100%|██████████| 1.66k/1.66k [00:00<00:00, 1.65MB/s]
Downloading: 100%|██████████| 235M/235M [00:09<00:00, 24.4MB/s] 


DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 205328
    })
})

In [6]:
from datasets import load_dataset
#show me some random examples
dataset = load_dataset("wikipedia", "20220301.simple")


{'id': '1',
 'url': 'https://simple.wikipedia.org/wiki/April',
 'title': 'April',
 'text': 'April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.\n\nApril always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.\n\nApril\'s flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.\n\nThe Month \n\nApril comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.\n\nApril begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other\'s last days are exactly 35 weeks (245 days) apart.\n\nIn co

In [8]:
dataset['train'][420]

{'id': '884',
 'url': 'https://simple.wikipedia.org/wiki/University',
 'title': 'University',
 'text': 'A university is a higher learning institution. The word university comes from the Latin universitas magistrorum et scholarium, roughly meaning "community of teachers and scholars".  Students can go to university to get an academic degree. Unlike the schooling they have done before, the courses at university are specialised. A person studying biology at university has many courses about biology and fewer courses in other fields such as languages or history. To get a higher degree, people must do some research.\n\nNot all subjects are offered at universities. Mainly, universities offer courses which are about knowledge. They usually do not offer courses in practical trades. In some cases such as law, where there are both knowledge and practical issues, the university does mainly the theoretical side of the subject. Practical qualifications are done elsewhere.\n\nHistory \n\nThe univers

In [ ]:
#lets procces the data
import os
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

def get_embedding(chunk):
    """Get the embedding vector for a chunk of text"""
    return model.encode(chunk)

def insert_docs_to_db(collection, batch_data):
    res = client.insert(
        collection_name=collection.name,  # Assuming collection has a 'name' attribute
        data=batch_data
    )
    return res

In [ ]:
import json
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

connections.connect(
  alias="default",
  host='localhost',
  port='19530'
)

# 2. Define fields
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384)
]

# 3. Build the schema
schema = CollectionSchema(
    fields,
    description="Schema of wikipedia collection",
        enable_dynamic_field=False
)

# 4. Create collection
collection = Collection(
    name="wiki_doc", 
    description="Wikipedia collection",
    schema=schema
)

# 5. Create index

index_params = {
    "index_type": "AUTOINDEX",
    "metric_type": "L2",
    "params": {}
}

# To name the index, do as follows:
collection.create_index(
  field_name="embedding", 
  index_params=index_params,
  index_name='embedding_index'
)